In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
netflix_titles = spark.read.format('csv').option('header','true').option('inferschema','true').load('abfss://bronze@adlsgen2datalake1245.dfs.core.windows.net/netflix_titles')

In [0]:
netflix_titles.display()

In [0]:
netflix_titles_filledna = netflix_titles.fillna({'duration_minutes': '0','duration_seasons': '0'})

In [0]:
netflix_titles_filledna.display()


In [0]:
netflix_convert = netflix_titles_filledna.withColumn('duration_minutes',col('duration_minutes').cast('integer')) \
    .withColumn('duration_seasons',col('duration_seasons').cast('integer'))
   

In [0]:
netflix_convert.display()

In [0]:
netflix_split = netflix_convert.withColumn('short_title',split(col('title'),':')[0])
netflix_split.display()

In [0]:
netflix_split_one = netflix_split.withColumn('rating',split(col('rating'),'-')[0])
netflix_split_one.display()



In [0]:
df = netflix_split_one.withColumn('typeflag',when(col('type')=='Movie',1).when(col('type')=='TV Show',2).otherwise(0))

In [0]:
df.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df_window = df.withColumn('duration_ranking',dense_rank().over(Window.orderBy(col('duration_minutes').desc())))


In [0]:
df_window.display()

In [0]:
test = df_window.groupBy('type').agg(count('*').alias('total_count'))
test.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_window.write.format('delta').option('header','true').mode('overwrite').option('path','abfss://silver@adlsgen2datalake1245.dfs.core.windows.net/netflix_titles').save()